In [1]:
import re
import requests
import pandas as pd
import numpy as np

#Country cleaner corrects any spelling mistakes in the countries.
from country_cleaner import country_clean


In [2]:
#### Importing csv data files into dataframes 
#####Erase stuff after pound before turning in.
df = pd.read_csv('Input_Data/education_expenditure_supplementary_data.csv')#Public and private direct expenditures on education institutions as a percentage of gross domestic product
df1 = pd.read_csv('Input_Data/educational_attainment_supplementary_data.csv')#refined list of survey questions by country ########country_name erase this line
df2 = pd.read_csv('Input_Data/school_and_country_table.csv')
df3 = pd.read_csv('Input_Data/shanghaiData.csv')####Not needed erase this
df4 = pd.read_csv('Input_Data/timesData.csv')
df_GDP = pd.read_csv('Input_Data/GDP.csv')####Country Name
df_GDP2 = pd.read_csv('Input_Data/GDP.csv')
higherED_df = pd.read_csv('Input_data/cwurData.csv')

### All DFS pass through Country Cleaner.
country_list = [df, df1, df2, df4, df_GDP, df_GDP2, higherED_df]

for frame in country_list:
    for col in frame.columns:
        if (col =='country_name') or (col == 'country') or (col == 'Country Name') or (col == 'Country'):
            frame[col] = frame[col].apply(lambda x: country_clean(x))
test_df = df1.loc[df1['country_name'] == 'Egypt']
test_df



,country_name,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
21675,Egypt,"Barro-Lee: Average years of primary schooling,...",NaN,1.65,NaN,NaN,2.02,NaN,NaN,NaN,...,3.21,NaN,NaN,NaN,NaN,3.47,NaN,NaN,NaN,NaN
21676,Egypt,"Barro-Lee: Average years of primary schooling,...",NaN,2.40,NaN,NaN,2.75,NaN,NaN,NaN,...,3.69,NaN,NaN,NaN,NaN,3.84,NaN,NaN,NaN,NaN
21677,Egypt,"Barro-Lee: Average years of primary schooling,...",NaN,3.18,NaN,NaN,3.77,NaN,NaN,NaN,...,4.22,NaN,NaN,NaN,NaN,4.56,NaN,NaN,NaN,NaN
21678,Egypt,"Barro-Lee: Average years of primary schooling,...",NaN,3.71,NaN,NaN,4.24,NaN,NaN,NaN,...,4.16,NaN,NaN,NaN,NaN,4.31,NaN,NaN,NaN,NaN
21679,Egypt,"Barro-Lee: Average years of primary schooling,...",NaN,2.60,NaN,NaN,3.00,NaN,NaN,NaN,...,4.96,NaN,NaN,NaN,NaN,4.03,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22095,Egypt,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22096,Egypt,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22097,Egypt,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22098,Egypt,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
### pound added so I can find this easier in GIT ###
#############################################################
### Cleaning Higher Ed Data.  We take the top nationally ranked colleges from each country and average the data
rank_one_df = higherED_df[higherED_df['national_rank'] == 1]


sorted_rank_df = rank_one_df.sort_values(['score','country'],ascending = (False,False))
drop_institution_df = sorted_rank_df.drop(columns=['institution','year'],axis =1)

###Swap columns
first_column = drop_institution_df.pop('country')
drop_institution_df.insert(0, 'country', first_column)

###Get mean and round to remove uncessary zeros.
mean_df = grouped_df = drop_institution_df.groupby(by=['country']).mean()
resorted_df = mean_df.sort_values(['score'],ascending=False).round(2)
h_ed_df= resorted_df.copy()




In [4]:
#Cleaning GDP file and averaging out years 2001 through 2011. This will help smooth out missing data and lag between school investments vs results 
df_GDP.rename({'Country Name': 'country'}, axis=1, inplace=True)
df_GDP['avg_2000_2011'] = df_GDP[['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010','2011']].mean(axis=1)
df_GDP = df_GDP[['country', 'Country Code','Indicator Name', 'Indicator Code','avg_2000_2011']]
df_GDP.dropna(subset=['avg_2000_2011'],inplace=True)
df_GDP.reset_index(inplace=True,drop=True)
df_GDP.head()


C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,country,Country Code,Indicator Name,Indicator Code,avg_2000_2011
0,World,WLD,GDP (current US$),NY.GDP.MKTP.CD,5.373832e+13
1,High income,HIC,GDP (current US$),NY.GDP.MKTP.CD,3.805390e+13
2,OECD members,OED,GDP (current US$),NY.GDP.MKTP.CD,3.787817e+13
3,Post-demographic dividend,PST,GDP (current US$),NY.GDP.MKTP.CD,3.594110e+13
4,IDA & IBRD total,IBT,GDP (current US$),NY.GDP.MKTP.CD,1.618787e+13


In [5]:
#Cleaning school expenditure file. This file is missing many countries (only has around 30)
#Filtering on school spend and creating dataframe for each by category - e.g. All, k_12, and hihger education
#We may be able to run test on all 3 of these groups 
df['institute_type'] = df['institute_type'].str.strip()
df['direct_expenditure_type'] = df['direct_expenditure_type'].str.strip()

df[['1995','2000','2005','2009','2010','2011']] = df[['1995','2000','2005','2009','2010','2011']]/100
df['school_spend_2000_2011'] = df[['2000','2005','2009', '2010','2011']].mean(axis=1)

df = df[df['direct_expenditure_type']== 'Total']
df = df[['country', 'institute_type','school_spend_2000_2011']]

df_all_edu = df[df['institute_type']=='All Institutions']
df_k_12_edu = df[df['institute_type']=='Elementary and Secondary Institutions']
df_higher_edu = df[df['institute_type']=='Higher Education Institutions']

df_all_edu = df_all_edu.dropna(subset=['school_spend_2000_2011'])
df_k_12_edu = df_k_12_edu.dropna(subset=['school_spend_2000_2011'])
df_higher_edu = df_higher_edu.dropna(subset=['school_spend_2000_2011'])

df_all_edu.reset_index(inplace=True,drop=True)
df_k_12_edu.reset_index(inplace=True,drop=True)
df_higher_edu.reset_index(inplace=True,drop=True)

df_all_edu.head()
df_k_12_edu.head()
df_higher_edu.head()

#df.institute_type.unique()

,country,institute_type,school_spend_2000_2011
0,OECD Average,Higher Education Institutions,0.016
1,Australia,Higher Education Institutions,0.016
2,Austria,Higher Education Institutions,0.015
3,Belgium,Higher Education Institutions,0.014
4,Chile,Higher Education Institutions,0.024


In [6]:
#Merging GDP and expenditure data. The column "school expenditures will be the $$$ spend on schooling"
df_all_edu_1 = df_GDP.merge(df_all_edu, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
df_k_12_edu_1 = df_GDP.merge(df_k_12_edu, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
df_higher_edu_1 = df_GDP.merge(df_higher_edu, on='country', how='left').dropna(subset=['school_spend_2000_2011'])

df_all_edu_1.reset_index(inplace=True,drop=True)
df_k_12_edu_1.reset_index(inplace=True,drop=True)
df_higher_edu_1.reset_index(inplace=True,drop=True)

df_all_edu_1['school_expenditures'] = round(df_all_edu_1['school_spend_2000_2011'] * df_all_edu_1['avg_2000_2011'],2)
df_k_12_edu_1['school_expenditures'] = round(df_k_12_edu_1['school_spend_2000_2011'] * df_k_12_edu_1['avg_2000_2011'],2)
df_higher_edu_1['school_expenditures'] = round(df_higher_edu_1['school_spend_2000_2011'] * df_higher_edu_1['avg_2000_2011'],2)

df_all_edu_1.head()
df_k_12_edu_1.head()
df_higher_edu_1.head()


,country,Country Code,Indicator Name,Indicator Code,avg_2000_2011,institute_type,school_spend_2000_2011,school_expenditures
0,United States of America,USA,GDP (current US$),NY.GDP.MKTP.CD,1.303684e+13,Higher Education Institutions,0.027,3.519947e+11
1,Japan,JPN,GDP (current US$),NY.GDP.MKTP.CD,4.944989e+12,Higher Education Institutions,0.016,7.911982e+10
2,Germany,DEU,GDP (current US$),NY.GDP.MKTP.CD,2.899084e+12,Higher Education Institutions,0.013,3.768809e+10
3,United Kingdom,GBR,GDP (current US$),NY.GDP.MKTP.CD,2.372315e+12,Higher Education Institutions,0.012,2.846777e+10
4,France,FRA,GDP (current US$),NY.GDP.MKTP.CD,2.206148e+12,Higher Education Institutions,0.015,3.309223e+10


In [7]:
### data cleaning the survey file to narrow in on questions we may want to take a look at. Column "series name"
df1 = df1[df1['Keep or not']==1]
df1['avg_2005_2015'] = df1[['2005', '2006','2007','2008','2009','2010','2011','2012','2013','2015']].mean(axis=1)
df1.dropna(subset=['avg_2005_2015'],inplace=True)
df1.reset_index(inplace=True,drop=True)
df1.head()

C:\Users\awnet\AppData\Local\Temp/ipykernel_22928/2786941589.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['avg_2005_2015'] = df1[['2005', '2006','2007','2008','2009','2010','2011','2012','2013','2015']].mean(axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,country_name,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2006,2007,2008,2009,2010,2011,2012,2013,2015,avg_2005_2015
0,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.50,NaN,NaN,0.66,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.97,NaN,NaN,NaN,NaN,1.620
1,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.57,NaN,NaN,1.87,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.85,NaN,NaN,NaN,NaN,3.530
2,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.16,NaN,NaN,NaN,NaN,1.080
3,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.23,NaN,NaN,NaN,NaN,2.915
4,Afghanistan,Barro-Lee: Percentage of population age 15+ wi...,1.0,77.80,NaN,NaN,75.04,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,58.38,NaN,NaN,NaN,NaN,62.055


In [8]:
#cleaning GDP file to use again. As many countries are missing expenditures, we can just use toal GDP vs school responses as well. This will be a broader dataset with more countries.
df_GDP2.rename({'Country Name': 'country'}, axis=1, inplace=True)
df_GDP2['avg_2000_2015'] = df_GDP2[['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010','2011','2012','2013','2014','2015']].mean(axis=1)
df_GDP2['avg_2000_2015'] = df_GDP2['avg_2000_2015']/1000000000
df_GDP2 = df_GDP2[['country', 'Country Code','Indicator Name', 'Indicator Code','avg_2000_2015']]
df_GDP2.dropna(subset=['avg_2000_2015'],inplace=True)
df_GDP2.reset_index(inplace=True,drop=True)
df_GDP2.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,country,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,World,WLD,GDP (current US$),NY.GDP.MKTP.CD,59517.199213
1,High income,HIC,GDP (current US$),NY.GDP.MKTP.CD,40900.271146
2,OECD members,OED,GDP (current US$),NY.GDP.MKTP.CD,40629.787880
3,Post-demographic dividend,PST,GDP (current US$),NY.GDP.MKTP.CD,38437.087014
4,IDA & IBRD total,IBT,GDP (current US$),NY.GDP.MKTP.CD,19195.578598


In [9]:
#import country_cleaner.country_clean as country_clean
#df_GDP2["Country Name"] = df_GDP2["Country Name"].apply(lambda x: country_clean(x))

In [10]:
# joinning survey data_sets with expenditures dataset for final data sets to beging running test on.
df1.rename({'country_name': 'country'}, axis=1, inplace=True)

complete_expenditures_all = df1.merge(df_all_edu_1, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
complete_expenditures_k_12 = df1.merge(df_k_12_edu_1, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
complete_expenditures_higher_edu = df1.merge(df_higher_edu_1, on='country', how='left').dropna(subset=['school_spend_2000_2011'])


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [11]:
#final data-set with total GDP (not expenditures) to run test on as it will have more datapoints vs above compplete tables
complete_GDP = df1.merge(df_GDP2, on='country', how='left').dropna(subset=['avg_2000_2015'])

In [12]:
complete_GDP.head(5)

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2010,2011,2012,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.50,NaN,NaN,0.66,NaN,NaN,NaN,...,1.97,NaN,NaN,NaN,NaN,1.620,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
1,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.57,NaN,NaN,1.87,NaN,NaN,NaN,...,3.85,NaN,NaN,NaN,NaN,3.530,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
2,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,1.16,NaN,NaN,NaN,NaN,1.080,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
3,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,3.23,NaN,NaN,NaN,NaN,2.915,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
4,Afghanistan,Barro-Lee: Percentage of population age 15+ wi...,1.0,77.80,NaN,NaN,75.04,NaN,NaN,NaN,...,58.38,NaN,NaN,NaN,NaN,62.055,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438


In [13]:
complete_GDP = complete_GDP[(complete_GDP['series_name']=='Barro-Lee: Average years of total schooling, age 25+, total') |(complete_GDP['series_name']=='Barro-Lee: Average years of total schooling, age 25+, female') | (complete_GDP['series_name']=='Barro-Lee: Percentage of population age 25+ with no education')]
complete_GDP.head()                         

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2010,2011,2012,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2015
2,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,1.16,NaN,NaN,NaN,NaN,1.080,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
3,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,3.23,NaN,NaN,NaN,NaN,2.915,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
5,Afghanistan,Barro-Lee: Percentage of population age 25+ wi...,1.0,84.62,NaN,NaN,80.70,NaN,NaN,NaN,...,66.02,NaN,NaN,NaN,NaN,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
8,Albania,"Barro-Lee: Average years of total schooling, a...",1.0,6.13,NaN,NaN,7.29,NaN,NaN,NaN,...,9.57,NaN,NaN,NaN,NaN,9.470,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655
9,Albania,"Barro-Lee: Average years of total schooling, a...",1.0,6.75,NaN,NaN,7.83,NaN,NaN,NaN,...,9.85,NaN,NaN,NaN,NaN,9.770,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655


In [14]:
df_pivot=complete_GDP.pivot(index=['country'], columns='series_name',values=['avg_2005_2015'])
df_pivot.columns = df_pivot.columns.droplevel()
modified_df=df_pivot.rename_axis(None,axis=1)
modified_df=modified_df.reset_index()
modified_df_final = modified_df.merge(df_GDP2, on='country', how='left').dropna(subset=['avg_2000_2015'])
modified_df_final

,country,"Barro-Lee: Average years of total schooling, age 25+, female","Barro-Lee: Average years of total schooling, age 25+, total",Barro-Lee: Percentage of population age 25+ with no education,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080
...,...,...,...,...,...,...,...,...
138,"Venezuela, RB",7.850,7.655,8.460,VEN,GDP (current US$),NY.GDP.MKTP.CD,245.228688
139,Vietnam,6.500,6.945,7.830,VNM,GDP (current US$),NY.GDP.MKTP.CD,96.776592
140,"Yemen, Rep.",1.065,2.230,73.060,YEM,GDP (current US$),NY.GDP.MKTP.CD,24.405175
141,Zambia,5.605,6.460,19.755,ZMB,GDP (current US$),NY.GDP.MKTP.CD,14.815902


In [15]:
modified_df_final.rename({'Barro-Lee: Average years of total schooling, age 25+, female': 'female_25_plus','Barro-Lee: Average years of total schooling, age 25+, total': 'All_25_plus','Barro-Lee: Percentage of population age 25+ with no education': 'All_25_plus_no_edu'}, axis=1, inplace=True)
modified_df_final.head()

,country,female_25_plus,All_25_plus,All_25_plus_no_edu,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080


In [16]:
modified_df_final['country_id'] = pd.factorize(modified_df_final.country)[0]

In [17]:
modified_df_final.head()

,country,female_25_plus,All_25_plus,All_25_plus_no_edu,Country Code,Indicator Name,Indicator Code,avg_2000_2015,country_id
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438,0
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655,1
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615,2
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498,3
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080,4


In [18]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer


X_complete_GDP = modified_df_final[['female_25_plus','All_25_plus','All_25_plus_no_edu','avg_2000_2015']]
y_complete_GDP = modified_df_final[['country_id']]
X_complete_GDP

,female_25_plus,All_25_plus,All_25_plus_no_edu,avg_2000_2015
0,1.080,2.915,68.800,12.328438
1,9.470,9.770,3.185,9.476655
2,4.835,5.570,31.295,133.920615
3,9.345,9.370,1.100,345.568498
4,10.795,10.835,0.490,7.306080
...,...,...,...,...
138,7.850,7.655,8.460,245.228688
139,6.500,6.945,7.830,96.776592
140,1.065,2.230,73.060,24.405175
141,5.605,6.460,19.755,14.815902


In [19]:
X_complete_GDP_normalized = MinMaxScaler().fit(X_complete_GDP).transform(X_complete_GDP)  
kmeans = KMeans(n_clusters = 5, random_state = 0)
kmeans.fit(X_complete_GDP)
kmeans.labels_
modified_df_final['cluster'] = kmeans.fit_predict(X_complete_GDP)
y_kmeans = kmeans.predict(X_complete_GDP)
kmeans.labels_
kmeans.cluster_centers_


array([[7.20852000e+00, 7.58896000e+00, 2.02831600e+01, 9.02871700e+01],
       [1.32750000e+01, 1.32750000e+01, 4.45000000e-01, 1.40743376e+04],
       [9.26300000e+00, 9.48800000e+00, 7.36900000e+00, 2.42373195e+03],
       [9.09200000e+00, 9.54350000e+00, 9.33150000e+00, 1.07619918e+03],
       [8.78000000e+00, 9.16750000e+00, 4.10000000e+00, 4.98019991e+03]])

In [20]:
h_ed_df.describe()

,world_rank,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score
count,59.000000,59.0,59.000000,59.000000,59.000000,59.00000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,353.546610,1.0,229.881017,282.851356,159.900847,369.37678,379.218475,330.740508,396.813559,448.558983,50.004576
std,309.691534,0.0,128.823050,200.200661,73.171126,318.43022,305.787634,242.381348,316.655691,295.852672,10.092858
min,1.000000,1.0,2.500000,3.000000,1.000000,1.00000,1.000000,1.000000,1.000000,4.250000,44.180000
25%,82.000000,1.0,104.625000,103.375000,105.250000,62.75000,118.085000,101.375000,105.250000,181.750000,44.790000
50%,291.500000,1.0,278.000000,213.500000,214.000000,259.50000,247.500000,268.500000,328.000000,483.500000,46.280000
75%,552.000000,1.0,361.000000,522.500000,214.000000,606.75000,683.500000,522.000000,746.250000,756.750000,49.780000
max,971.500000,1.0,361.000000,522.500000,214.000000,998.50000,962.000000,806.000000,956.000000,804.000000,100.000000


In [21]:
h_ed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, United States of America to Puerto Rico
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            59 non-null     float64
 1   national_rank         59 non-null     float64
 2   quality_of_education  59 non-null     float64
 3   alumni_employment     59 non-null     float64
 4   quality_of_faculty    59 non-null     float64
 5   publications          59 non-null     float64
 6   influence             59 non-null     float64
 7   citations             59 non-null     float64
 8   broad_impact          59 non-null     float64
 9   patents               59 non-null     float64
 10  score                 59 non-null     float64
dtypes: float64(11)
memory usage: 5.5+ KB


In [22]:
h_ed_df.corr()

,world_rank,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score
world_rank,1.000000,NaN,0.597069,0.755904,0.627145,0.920100,0.862065,0.858061,0.908768,0.729807,-0.536188
national_rank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
quality_of_education,0.597069,NaN,1.000000,0.446030,0.845748,0.562158,0.570039,0.548860,0.547861,0.434614,-0.590930
alumni_employment,0.755904,NaN,0.446030,1.000000,0.442705,0.601342,0.549113,0.537933,0.568028,0.561378,-0.539092
quality_of_faculty,0.627145,NaN,0.845748,0.442705,1.000000,0.601272,0.655108,0.646750,0.627308,0.531992,-0.662818
publications,0.920100,NaN,0.562158,0.601342,0.601272,1.000000,0.891539,0.900068,0.953361,0.672824,-0.517084
influence,0.862065,NaN,0.570039,0.549113,0.655108,0.891539,1.000000,0.915437,0.944099,0.696907,-0.526425
citations,0.858061,NaN,0.548860,0.537933,0.646750,0.900068,0.915437,1.000000,0.937196,0.642703,-0.527331
broad_impact,0.908768,NaN,0.547861,0.568028,0.627308,0.953361,0.944099,0.937196,1.000000,0.675042,-0.521284
patents,0.729807,NaN,0.434614,0.561378,0.531992,0.672824,0.696907,0.642703,0.675042,1.000000,-0.564994


In [23]:
h_ed_df.cov()

,world_rank,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score
world_rank,95908.846416,0.0,23820.292993,46866.371512,14211.410827,90735.795091,81637.409124,64409.036355,89118.831771,66867.167869,-1675.949146
national_rank,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
quality_of_education,23820.292993,0.0,16595.378209,11503.309059,7972.124834,23060.359550,22455.256074,17137.760465,22348.646745,16564.282570,-768.322884
alumni_employment,46866.371512,0.0,11503.309059,40080.304746,6485.141842,38335.507944,33616.094821,26103.150523,36009.923447,33250.367970,-1089.287401
quality_of_faculty,14211.410827,0.0,7972.124834,6485.141842,5354.013711,14009.570053,14657.925958,11470.316715,14534.750678,11516.500709,-489.495313
publications,90735.795091,0.0,23060.359550,38335.507944,14009.570053,101397.805312,86810.999085,69468.620596,96130.005855,63385.721833,-1661.839933
influence,81637.409124,0.0,22455.256074,33616.094821,14657.925958,86810.999085,93506.076968,67849.676080,91416.566262,63047.874195,-1624.689070
citations,64409.036355,0.0,17137.760465,26103.150523,11470.316715,69468.620596,67849.676080,58748.717629,71931.120872,46087.685545,-1290.021742
broad_impact,89118.831771,0.0,22348.646745,36009.923447,14534.750678,96130.005855,91416.566262,71931.120872,100270.826710,63240.236876,-1666.002839
patents,66867.167869,0.0,16564.282570,33250.367970,11516.500709,63385.721833,63047.874195,46087.685545,63240.236876,87528.803468,-1687.071206


In [24]:
modified_df_final.head()


,country,female_25_plus,All_25_plus,All_25_plus_no_edu,Country Code,Indicator Name,Indicator Code,avg_2000_2015,country_id,cluster
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438,0,0
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655,1,0
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615,2,0
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498,3,0
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080,4,0


In [25]:
modified_df_final.describe()

,female_25_plus,All_25_plus,All_25_plus_no_edu,avg_2000_2015,country_id,cluster
count,143.000000,143.000000,143.000000,143.000000,143.00000,143.000000
mean,7.476469,7.853881,18.700699,407.001995,71.00000,0.342657
std,3.384885,3.094401,21.407629,1376.958261,41.42463,0.942763
min,0.730000,1.175000,0.020000,0.318162,0.00000,0.000000
25%,4.910000,5.680000,2.427500,11.207324,35.50000,0.000000
50%,8.095000,7.895000,8.460000,39.536689,71.00000,0.000000
75%,10.235000,10.585000,30.085000,219.227647,106.50000,0.000000
max,13.275000,13.275000,80.600000,14074.337562,142.00000,4.000000


In [27]:
h_ed_df.reset_index(level=0, inplace=True)

In [30]:
a_final_frame = pd.concat([h_ed_df,modified_df_final], axis=1,join="inner")
a_final_frame

,index,country,world_rank,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,...,country,female_25_plus,All_25_plus,All_25_plus_no_edu,Country Code,Indicator Name,Indicator Code,avg_2000_2015,country_id,cluster
0,0,United States of America,1.00,1.0,2.50,3.00,1.00,1.00,1.00,1.00,...,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438,0,0
1,1,United Kingdom,15.00,1.0,18.40,31.40,13.40,22.40,15.80,23.60,...,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655,1,0
2,2,Japan,13.50,1.0,20.50,7.00,33.50,12.00,19.25,27.75,...,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615,2,0
3,3,Switzerland,18.00,1.0,18.75,73.75,13.00,41.50,39.75,45.75,...,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498,3,0
4,4,Israel,21.00,1.0,12.25,155.25,17.00,109.00,88.50,265.75,...,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080,4,0
5,5,Canada,31.75,1.0,83.25,69.50,33.75,3.25,15.50,13.00,...,Australia,11.645,11.655,0.920,AUS,GDP (current US$),NY.GDP.MKTP.CD,939.856047,5,3
6,6,South Korea,40.75,1.0,231.00,16.25,157.50,41.50,132.50,94.50,...,Austria,8.775,9.685,1.500,AUT,GDP (current US$),NY.GDP.MKTP.CD,345.472946,6,0
7,7,France,38.75,1.0,45.00,171.75,81.50,208.75,166.75,232.25,...,Bahrain,6.805,6.830,32.015,BHR,GDP (current US$),NY.GDP.MKTP.CD,21.182949,7,0
8,8,China,55.50,1.0,268.50,36.50,214.00,58.50,143.00,216.00,...,Bangladesh,4.035,4.550,44.450,BGD,GDP (current US$),NY.GDP.MKTP.CD,99.847685,8,0
9,9,Taiwan,61.00,1.0,102.50,24.50,214.00,46.00,219.50,232.50,...,Barbados,9.370,9.255,0.020,BRB,GDP (current US$),NY.GDP.MKTP.CD,4.107625,9,0
